<a href="https://colab.research.google.com/github/snehas3120/Coins-Classification/blob/main/coinsclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
import glob

%matplotlib 

Using matplotlib backend: agg


In [61]:
input_path = Path('/content/images')

im_size = 320

In [62]:

image_files =list(input_path.glob('*.jpg'))
print(image_files)

[PosixPath('/content/images/5_1477146756.jpg'), PosixPath('/content/images/5_1477146750.jpg'), PosixPath('/content/images/25_1477153962.jpg'), PosixPath('/content/images/10_1477289196.jpg'), PosixPath('/content/images/10_1477288368.jpg'), PosixPath('/content/images/5_1477189500.jpg'), PosixPath('/content/images/25_1477149450.jpg'), PosixPath('/content/images/10_1477289004.jpg'), PosixPath('/content/images/5_1477146954.jpg'), PosixPath('/content/images/25_1477149510.jpg'), PosixPath('/content/images/5_1477289898.jpg'), PosixPath('/content/images/25_1477192020.jpg'), PosixPath('/content/images/5_1477189146.jpg'), PosixPath('/content/images/5_1477147104.jpg'), PosixPath('/content/images/25_1477191576.jpg'), PosixPath('/content/images/5_1477290114.jpg'), PosixPath('/content/images/10_1477288500.jpg'), PosixPath('/content/images/5_1477290486.jpg'), PosixPath('/content/images/10_1477289178.jpg'), PosixPath('/content/images/5_1477290876.jpg'), PosixPath('/content/images/10_1477288746.jpg'), P

In [66]:
def read_file(fname):
    # Read image
    im = Image.open(fname)

    # Resize
    im.thumbnail((im_size, im_size))

    # Convert to numpy array
    im_array = np.asarray(im)

    # Get target
    target = int(fname.stem.split('_')[0])

    return im_array, target

In [67]:

images = []
targets = []

for image_file  in  tqdm(image_files):

    
    image, target = read_file(image_file)
    
    images.append(image)
    targets.append(target)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [68]:
X = (np.array(images).astype(np.float32) / 127.5) - 1
y_cls = np.array(targets)

In [69]:
X.shape, y_cls.shape

((1671, 240, 320, 3), (1671,))

In [70]:
coins_ids = {
    5: 0,
    10: 1,
    25: 2,
    50: 3,
    100: 4
}

ids_coins = [5, 10, 25, 50, 100]

y = np.array([coins_ids.get(coin) for coin in y_cls])
print(y)

[0 0 2 ... 1 2 1]


In [71]:
X_train, X_valid, y_train, y_valid, fname_train, fname_valid = train_test_split(
    X, y, image_files, test_size=0.2, random_state=42)

In [72]:
im_width = X.shape[2]
im_height = X.shape[1]

im_width, im_height

(320, 240)

In [73]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, GlobalAvgPool2D, GlobalMaxPool2D, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [74]:
model = Sequential()
# CNN network
model.add( Conv2D(16, 3, activation='relu', padding='same', input_shape=(im_height, im_width, 3)) )
model.add( MaxPool2D(2) )

model.add( Conv2D(32, 3, activation='relu', padding='same') )
model.add( MaxPool2D(2) )

model.add( Conv2D(64, 3, activation='relu', padding='same') )
model.add( MaxPool2D(2) )

model.add( Conv2D(128, 3, activation='relu', padding='same') )
model.add( MaxPool2D(2) )

model.add( Conv2D(256, 3, activation='relu', padding='same') )

# Transition between CNN and MLP
model.add( GlobalAvgPool2D() )

# MLP network
model.add( Dense(256, activation='relu') )

model.add( Dense(5, activation='softmax') )

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 240, 320, 16)      448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 120, 160, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 120, 160, 32)      4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 60, 80, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 60, 80, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 40, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 30, 40, 128)      

In [75]:
optim = Adam(lr=1e-3)
model.compile(optim, 'sparse_categorical_crossentropy', metrics=['acc'])
callbacks = [
    ReduceLROnPlateau(patience=5, factor=0.1, verbose=True),
    ModelCheckpoint('best.model', save_best_only=True),
    EarlyStopping(patience=12)
]

history = model.fit(X_train, y_train, epochs=2000, validation_data=(X_valid, y_valid), batch_size=32,
                   callbacks=callbacks)
model.evaluate(X_valid, y_valid)

y_pred = model.predict(X_valid)
y_pred_cls = y_pred.argmax(1)
errors = np.where(y_pred_cls != y_valid)[0]
errors

Epoch 1/2000
42/42 [==============================] - 121s 3s/step - loss: 1.3503 - acc: 0.3067 - val_loss: 1.1873 - val_acc: 0.3493
INFO:tensorflow:Assets written to: best.model/assets
Epoch 2/2000
42/42 [==============================] - 131s 3s/step - loss: 1.1986 - acc: 0.3209 - val_loss: 1.1880 - val_acc: 0.3881
Epoch 3/2000
42/42 [==============================] - 122s 3s/step - loss: 1.1736 - acc: 0.3717 - val_loss: 1.1815 - val_acc: 0.3493
INFO:tensorflow:Assets written to: best.model/assets
Epoch 4/2000
42/42 [==============================] - 126s 3s/step - loss: 1.2008 - acc: 0.3573 - val_loss: 1.1703 - val_acc: 0.3791
INFO:tensorflow:Assets written to: best.model/assets
Epoch 5/2000
42/42 [==============================] - 122s 3s/step - loss: 1.1763 - acc: 0.3718 - val_loss: 1.1269 - val_acc: 0.5015
INFO:tensorflow:Assets written to: best.model/assets
Epoch 6/2000
42/42 [==============================] - 122s 3s/step - loss: 1.1853 - acc: 0.4227 - val_loss: 1.1113 - val_ac

array([  4,   8,   9,  11,  39,  77,  88,  96,  99, 262, 267])

In [82]:
print("y_pred",y_pred)
print("y_pred_class",y_pred_cls)
from sklearn.metrics import classification_report
print(classification_report(y_valid,y_pred_cls))

y_pred [[2.8296982e-04 9.9971539e-01 3.0809242e-09 3.3730963e-08 1.6132378e-06]
 [9.4955607e-04 9.9904734e-01 7.1592865e-09 2.7556231e-08 3.0886870e-06]
 [9.8547840e-01 5.4671657e-03 7.4221846e-03 4.0881819e-04 1.2234004e-03]
 ...
 [6.0703547e-04 9.9939072e-01 1.8402082e-09 1.1669760e-08 2.3099035e-06]
 [9.9999952e-01 1.9421613e-10 4.6659417e-07 1.0225318e-14 2.8407281e-13]
 [9.9965262e-01 9.0067319e-05 2.5109533e-04 1.7281934e-08 6.1576261e-06]]
y_pred_class [1 1 0 1 0 0 0 1 0 3 2 1 0 2 1 0 2 0 1 1 2 1 2 0 1 1 0 2 0 2 0 2 0 0 2 1 0
 1 3 2 1 0 1 0 0 0 0 2 1 2 0 1 2 2 0 1 0 2 0 1 1 1 2 2 1 2 2 0 0 2 2 0 1 1
 2 1 2 3 1 2 1 1 1 0 1 1 1 0 2 1 1 0 0 0 2 0 1 2 1 3 2 1 0 2 1 0 2 1 0 2 1
 1 2 2 2 2 0 2 2 1 1 2 1 2 1 0 0 1 2 2 2 1 0 1 2 0 2 2 2 1 1 1 1 2 0 0 0 1
 0 0 1 0 2 0 0 1 1 0 0 2 1 2 3 1 1 1 1 1 1 2 2 1 3 0 1 1 0 1 2 1 1 1 0 0 1
 1 1 0 1 1 2 0 0 0 1 1 1 1 0 2 2 1 0 0 1 2 2 0 1 0 0 2 0 2 1 0 1 2 0 0 1 2
 0 0 0 1 0 0 0 1 0 0 1 0 2 2 4 1 0 2 1 0 1 2 2 2 0 1 1 2 1 0 2 1 0 0 1 1 2
 1 0 2 0 1 